In [1]:
import Deproject_v1_0 
from Deproject_test import *
from Deproject_plots import *
from scipy import sparse as scisp
from scipy.optimize import approx_fprime
import os
import cProfile


In [2]:
import importlib
importlib.reload(Deproject_v1_0)

<module 'Deproject_v1_0' from '/Users/mcmillan/code/DeprojectionProject/Deproject_v1_0.py'>

In [3]:

import astropy.units as u
import astropy.coordinates as coord

In [4]:
coord.Galactic()

<Galactic Frame>

In [5]:
N = 5000 #Number of stars we want to use in our sample
sample = Deproject_v1_0.model_sample(N)

In [6]:
sample

<Galactic Coordinate: (l, b, distance) in (deg, deg, pc)
    [(130.46222971,  34.03124939, 87.25725524),
     ( 52.05489427, -10.70637777, 65.28901972),
     (249.43870209,  -1.3788577 , 56.60858133), ...,
     (219.17527975, -17.29714562, 53.97803709),
     ( 70.12076943, -50.21294881, 66.60946959),
     ( 68.20349066,  34.29943796, 63.54261149)]
 (pm_l_cosb, pm_b, radial_velocity) in (mas / yr, mas / yr, km / s)
    [( -7.86542483,   10.63455611,  -9.5110396 ),
     ( 84.56772761,  -11.20543461, -17.53919632),
     ( 61.71977098,  135.92184691, -13.44703375), ...,
     (-66.99310988, -233.31046822,  15.18666022),
     (-49.31811174,   37.49680773, -19.63742809),
     ( 44.70008973,  -68.64585025, -29.01361751)]>

In [7]:

importlib.reload(Deproject_v1_0)

#sample = Deproject_v1_0.model_sample(N)
pvals, rhatvals = Deproject_v1_0.calc_p_rhat(sample)
sigma2, vmean0 = Deproject_v1_0.calc_sigma2(pvals,rhatvals,True)
np.sqrt(sigma2)

array([26.50470429, 28.80206576, 28.57058376])

In [10]:

n = np.array([6,6,6])
dv = np.array([20,20,20])
vmin = np.array([-60,-60,-60])

In [11]:

sigma2, vmean0 = Deproject_v1_0.calc_sigma2(pvals,rhatvals,True)

sigma = np.sqrt(sigma2)
phi0 = phi_guess(vmean0,sigma,vmin,dv,n)

sanity_check(pvals,rhatvals,phi0,vmin,dv,n)

Sample mean ===> [ 8.52194543 -1.14849784  8.91541818]
Sample velocity dispersion ===> [26.50470429 28.80206576 28.57058376]
Computed mean ===> [ 7.31911092 -0.92952765  7.20930209]
Computed velocity dispersion ===> [24.37335997 25.90783727 25.48361687]


In [68]:
def compute_sec_der_local(phi_in,sigma2,dv) : 
    sec_der = np.zeros(phi_in.shape)
    sec_der[1:-1,:,:] += (phi_in[:-2,:,:] + phi_in[2:,:,:] - 2.*phi_in[1:-1,:,:]) * sigma2[0]/(dv[0])
    sec_der[:,1:-1,:] += (phi_in[:,:-2,:] + phi_in[:,2:,:] - 2.*phi_in[:,1:-1,:]) * sigma2[1]/(dv[1])
    sec_der[:,:,1:-1] += (phi_in[:,:,:-2] + phi_in[:,:,2:] - 2.*phi_in[:,:,1:-1]) * sigma2[2]/(dv[2])
    return sec_der

def compute_sec_der_sq_local(phi_in,sigma2,dv) :
    sec_der_sq = compute_sec_der_local(phi_in,sigma2,dv)**2
    return np.sum(sec_der_sq)

In [69]:
def compute_grad_sec_der_sq(phi_in,sigma2,dv) :
    grad_sec_der_sq = np.zeros(phi0.shape)
    sec_der_loc = compute_sec_der_local(phi0,sigma2,dv)

    grad_sec_der_sq[:-2,:,:] += 2 * (sec_der_loc[1:-1,:,:]) * sigma2[0]/(dv[0])
    grad_sec_der_sq[2:,:,:] += 2 * (sec_der_loc[1:-1,:,:])  * sigma2[0]/(dv[0])
    grad_sec_der_sq[1:-1,:,:] += -4 * (sec_der_loc[1:-1,:,:]) * sigma2[0]/(dv[0])


    grad_sec_der_sq[:,:-2,:] += 2 * (sec_der_loc[:,1:-1,:]) * sigma2[1]/(dv[1])
    grad_sec_der_sq[:,2:,:] += 2 * (sec_der_loc[:,1:-1,:]) * sigma2[1]/(dv[1])
    grad_sec_der_sq[:,1:-1,:] += -4 * (sec_der_loc[:,1:-1,:]) * sigma2[1]/(dv[1])

    grad_sec_der_sq[:,:,:-2] += 2 * (sec_der_loc[:,:,1:-1]) * sigma2[2]/(dv[2])
    grad_sec_der_sq[:,:,2:] += 2 * (sec_der_loc[:,:,1:-1]) * sigma2[2]/(dv[2])
    grad_sec_der_sq[:,:,1:-1] += -4 * (sec_der_loc[:,:,1:-1]) * sigma2[2]/(dv[2])
    return grad_sec_der_sq

# Check by comparison to numerical derivative

In [77]:
epsilon = np.zeros(phi0.shape)

small = 1e-8

grad_sec_der_sq = compute_grad_sec_der_sq(phi0,sigma2,dv)

for ii in range(6) :
    for jj in range(6) :
        for kk in range(6) :
            epsilon = np.zeros(phi0.shape)
            epsilon[ii,jj,kk] = small
            grad_num = (compute_sec_der_sq_local(phi0+epsilon,sigma2,dv) 
                        - compute_sec_der_sq_local(phi0,sigma2,dv) )/ small
            print(ii,jj,kk,':  ', grad_num - grad_sec_der_sq[ii,jj,kk])

0 0 0 :   0.007631149750523036
0 0 1 :   0.0023402095766869024
0 0 2 :   0.0027670621557263075
0 0 3 :   0.002767062181192159
0 0 4 :   0.002340209502108337
0 0 5 :   0.0018103836828231579
0 1 0 :   0.004216751487547299
0 1 1 :   0.000409684076657868
0 1 2 :   0.0012633893866222934
0 1 3 :   0.0012633893102247384
0 1 4 :   0.0004096842358194408
0 1 5 :   -0.0016040146419982193
0 2 0 :   0.005958345447652391
0 2 1 :   -0.0019278939435025677
0 2 2 :   -0.0010741886735559092
0 2 3 :   -0.0010741887708718423
0 2 4 :   0.0038928721860429505
0 2 5 :   0.0001375793417537352
0 3 0 :   0.005958345447652391
0 3 1 :   0.0038928721241973108
0 3 2 :   -0.001074188695383782
0 3 3 :   -0.0010741887817857787
0 3 4 :   0.0038928721860429505
0 3 5 :   0.0001375793417537352
0 4 0 :   0.004216751463900437
0 4 1 :   0.00040968414941744413
0 4 2 :   0.0012633894339160179
0 4 3 :   0.001263389323867159
0 4 4 :   0.006230450316252245
0 4 5 :   0.004216751449348521
0 5 0 :   0.0018103836646332638
0 5 1 :   0.0

# Below commented out (by turning to markdown)

alpha_guess_large = 0.

v_guess = np.array([1,1,1])
disp_guess = np.array([22,17,30])
import time
tin = time.time()
phi_hs, phiall = max_L(alpha_guess_large, pvals, rhatvals, vmin, 
                       dv, n, v0_guess = v_guess, disp_guess = disp_guess)
# cProfile.run('max_L(8e-14, pvals, rhatvals, vmin, dv, n,v0_guess=v_guess, disp_guess=disp_guess)')

print(time.time()-tin)
sanity_check(pvals,rhatvals,phi_hs,vmin,dv,n)



plot_fv(phi_hs,'xz',vmin,dv,n)

fv = np.sum(np.exp(phi_hs),axis=2)
plt.imshow(fv.T,origin='lower')
plt.show()